<a href="https://www.kaggle.com/code/llkh0a/checkpoint2?scriptVersionId=244412039" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🌎 Amazon Archaeological Discovery Pipeline

## Summary
Automated detection of potential archaeological sites using GEDI LiDAR and TerraBrasilis data, powered by GPT-4 analysis.

## Key Achievements ✅
- **Data**: GEDI L2B LiDAR + TerraBrasilis deforestation maps
- **Output**: 5 candidate sites with bbox WKT coordinates (±50m accuracy)
- **Validation**: Deforestation overlay analysis
- **AI Integration**: GPT-4 archaeological assessment

## Stats 📊
- Auto-filtered data points
- Reproducible anomaly detection
- JSON-logged analysis pipeline
- Validated site coordinates

## Stack 🛠️
- Python (GeoPandas, NumPy)
- OpenAI GPT-4
- GEDI LiDAR processing
- Spatial analysis tools

📍 Last Run: May 21, 2025

In [ ]:
%ls /kaggle/input/

In [ ]:
# Check the contents of the Kaggle input directory
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f'{subindent}{f}')

print("Contents of /kaggle/input directory:")
list_files('/kaggle/input')

In [ ]:
!pip install h5py geopandas

In [ ]:
!pip install scikit-learn

In [ ]:
import h5py
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import warnings
import json
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
# On Kaggle, input files are in /kaggle/input
gedi_path = '/kaggle/input/gedi02-and-terrabrasilis/GEDI02_B_2019108093620_O01965_01_T05338_02_003_01_V002.h5'
terrabrasilis_path = '/kaggle/input/gedi02-and-terrabrasilis/deforestation/prodes_amazonia_nb.gpkg'


In [ ]:
!pip install openai


In [ ]:

from openai import OpenAI
from datetime import datetime
os.environ["OPENAI_API_KEY"] = "<REDACTED>"


# Function to log prompts and responses
def log_openai_interaction(prompt, response, log_file='openai_logs.json'):
    log_entry = {
        'timestamp': datetime.now().isoformat(),
        'prompt': prompt,
        'response': response
    }
    
    try:
        with open(log_file, 'r') as f:
            logs = json.load(f)
    except FileNotFoundError:
        logs = []
    
    logs.append(log_entry)
    
    with open(log_file, 'w') as f:
        json.dump(logs, f, indent=4)


In [ ]:
def print_hdf5_structure(filepath):
    try:
        with h5py.File(filepath, 'r') as f:
            def print_attrs(name, obj):
                print(name)
                for key, val in obj.attrs.items():
                    print(f"    Attribute: {key}: {val}")
            
            f.visititems(print_attrs)
            
    except Exception as e:
        print(f"Error examining HDF5 file: {str(e)}")

print("\nExamining GEDI HDF5 file structure:")
if os.path.exists(gedi_path):
    print(f"File exists at: {gedi_path}")
    print_hdf5_structure(gedi_path)
else:
    print(f"File not found at: {gedi_path}")

In [ ]:
# Let's examine the structure of the first beam
print("\nExamining detailed BEAM structure:")
with h5py.File(gedi_path, 'r') as f:
    beams = [g for g in f.keys() if g.startswith('BEAM')]
    if beams:
        beam = beams[0]
        print(f"\nContents of {beam}:")
        for key in f[beam].keys():
            print(f"  - {key}")
            if isinstance(f[beam][key], h5py.Group):
                for subkey in f[beam][key].keys():
                    print(f"    - {subkey}")

In [ ]:
# Check if files exist
print(f"GEDI file exists: {os.path.exists(gedi_path)}")
print(f"TerrasBrasilis file exists: {os.path.exists(terrabrasilis_path)}")

if not os.path.exists(gedi_path):
    print(f"Looking for GEDI file at: {gedi_path}")

if not os.path.exists(terrabrasilis_path):
    print(f"Looking for TerrasBrasilis file at: {terrabrasilis_path}")

# Data Loading and Preparation

1. Load GEDI L2B data (contains canopy cover and vertical profile metrics)
2. Load TerrasBrasilis deforestation data
3. Process and prepare data for anomaly detection

In [ ]:
# Load GEDI data
def load_gedi_data(filepath):
    with h5py.File(filepath, 'r') as f:
        # First check the beams
        beamNames = [g for g in f.keys() if g.startswith('BEAM')]
        print(f"Available beams: {beamNames}")
        
        # Initialize lists to store data from all beams
        all_latitudes = []
        all_longitudes = []
        all_quality_flags = []
        all_degraded_flags = []
        all_pai = []
        all_sensitivity = []
        all_beam_ids = []
        all_rh100 = []
        # Collect data from each beam
        for beam in beamNames:
            print(f"Processing {beam} - {f[beam].attrs['description']}")
            
            try:
                # Required fields
                latitude = f[f'{beam}/geolocation/latitude_bin0'][:]
                longitude = f[f'{beam}/geolocation/longitude_bin0'][:]
                quality_flag = f[f'{beam}/l2b_quality_flag'][:]
                degrade_flag = f[f'{beam}/geolocation/degrade_flag'][:]
                pai = f[f'{beam}/pai'][:]
                sensitivity = f[f'{beam}/sensitivity'][:]
                rh_100 = f[f'{beam}/rh100'][:]
                # Append to lists
                all_latitudes.extend(latitude)
                all_longitudes.extend(longitude)
                all_quality_flags.extend(quality_flag)
                all_pai.extend(pai)
                all_degraded_flags.extend(degrade_flag)
                all_sensitivity.extend(sensitivity)
                all_beam_ids.extend([beam] * len(latitude))
                all_rh100.extend(rh_100)
                
            except Exception as e:
                print(f"Error processing {beam}: {str(e)}")
                continue
        
        # Create DataFrame with all beams
        data = pd.DataFrame({
            'latitude': all_latitudes,
            'longitude': all_longitudes,
            'quality_flag': all_quality_flags,
            'pai': all_pai,
            'degrade_flag': all_degraded_flags,
            'sensitivity': all_sensitivity,
            'beam': all_beam_ids,
            'rh100': all_rh100
        })
        
        return data

# Load GEDI data
try:
    print("Loading GEDI data...")
    print(f"File path: {gedi_path}")
    print(f"File exists: {os.path.exists(gedi_path)}")
    
    gedi_data = load_gedi_data(gedi_path)
    print(f"\nLoaded GEDI data shape: {gedi_data.shape}")
    print("\nGEDI data sample:")
    print(gedi_data.head())
    
    # Print beam statistics
    print("\nData points per beam:")
    print(gedi_data['beam'].value_counts())
except Exception as e:
    print(f"Error loading GEDI data: {str(e)}")
    print(f"Error type: {type(e)}")
    import traceback
    traceback.print_exc()

In [ ]:
gedi_data.describe()

In [ ]:
gedi_data.head()

In [ ]:
# Load TerrasBrasilis deforestation data
def load_terrabrasilis_data(filepath):
    try:
        # Load the data
        data = gpd.read_file(filepath)
        
        # Basic data validation
        print("\nTerrasBrasilis Data Info:")
        print("-" * 40)
        print(f"Number of records: {len(data)}")
        print(f"Columns: {data.columns.tolist()}")
        print(f"CRS: {data.crs}")
        
        # Check for missing values
        nulls = data.isnull().sum()
        if nulls.any():
            print("\nMissing values found:")
            print(nulls[nulls > 0])
            
        # Verify geometry validity
        invalid_geoms = data[~data.geometry.is_valid]
        if len(invalid_geoms) > 0:
            print(f"\nWarning: Found {len(invalid_geoms)} invalid geometries")
            # Try to fix invalid geometries
            data.geometry = data.geometry.buffer(0)
        
        return data
        
    except Exception as e:
        print(f"Error loading TerrasBrasilis data: {str(e)}")
        print(f"Error type: {type(e)}")
        traceback.print_exc()
        return None

# Load the data
deforestation_data = load_terrabrasilis_data(terrabrasilis_path)
if deforestation_data is not None:
    print("\nTerrasBrasilis data sample:")
    print(deforestation_data.head())

In [ ]:

deforestation_data.describe()

In [ ]:
deforestation_data.head(5)

## Anomaly Detection Pipeline

1. Preprocess GEDI data:
   - Filter by quality flags and sensitivity
   - Create spatial features
   - Remove invalid measurements

2. Calculate anomaly scores:
   - Height-based anomalies using rh100
   - Vegetation density anomalies using PAI
   - Combined anomaly score

3. Generate candidate footprints:
   - Identify top anomalies
   - Create bounding boxes
   - Validate with deforestation data

In [ ]:
# Preprocess GEDI data
def preprocess_gedi_data(data):
    # Remove invalid or poor quality data
    cleaned_data = data[
        (data['quality_flag'] == 1) &      # High quality flags
        (data['degrade_flag'] == 0) &      # No degradation
        (data['sensitivity'] > 0.95) &     # High sensitivity
        (data['rh100'] > 0) &             # Valid canopy heights
        (data['pai'] > 0)                 # Valid Plant Area Index
    ].copy()
    
    # Add derived features
    cleaned_data['shot_id'] = range(len(cleaned_data))  # Unique identifier for each measurement
    
    # Convert to GeoDataFrame for spatial analysis
    geometry = [Point(xy) for xy in zip(cleaned_data['longitude'], cleaned_data['latitude'])]
    cleaned_data = gpd.GeoDataFrame(cleaned_data, geometry=geometry, crs="EPSG:4326")
    
    return cleaned_data

# Preprocess the data
print("Preprocessing GEDI data...")
processed_gedi = preprocess_gedi_data(gedi_data)
print(f"\nOriginal data shape: {gedi_data.shape}")
print(f"Processed data shape: {processed_gedi.shape}")
print("\nProcessed data sample:")
print(processed_gedi.head())

In [ ]:
# Calculate anomaly scores
def calculate_anomaly_scores(data, window_size=5):
    # Calculate local statistics for height
    data['local_mean_height'] = data.groupby('beam')['rh100'].rolling(
        window=window_size, center=True
    ).mean().reset_index(0, drop=True)
    
    data['local_std_height'] = data.groupby('beam')['rh100'].rolling(
        window=window_size, center=True
    ).std().reset_index(0, drop=True)
    
    # Calculate height anomaly score
    data['height_anomaly_score'] = np.abs(
        (data['rh100'] - data['local_mean_height']) / 
        data['local_std_height']
    )
    
    # Calculate PAI anomaly score
    data['local_mean_pai'] = data.groupby('beam')['pai'].rolling(
        window=window_size, center=True
    ).mean().reset_index(0, drop=True)
    
    data['local_std_pai'] = data.groupby('beam')['pai'].rolling(
        window=window_size, center=True
    ).std().reset_index(0, drop=True)
    
    data['pai_anomaly_score'] = np.abs(
        (data['pai'] - data['local_mean_pai']) / 
        data['local_std_pai']
    )
    
    # Combined anomaly score
    data['combined_anomaly_score'] = (
        data['height_anomaly_score'] + 
        data['pai_anomaly_score']
    ) / 2
    
    return data.fillna(0)

# Calculate anomaly scores
print("Calculating anomaly scores...")
analysis_data = calculate_anomaly_scores(processed_gedi)
print("\nAnomalies calculated. Sample results:")
print(analysis_data[['rh100', 'pai', 'height_anomaly_score', 'pai_anomaly_score', 'combined_anomaly_score']].describe())

In [ ]:
# Generate candidate footprints
def generate_candidate_footprints(data, n_candidates=5, buffer_degrees=0.01):
    # Sort by combined anomaly score
    top_anomalies = data.nlargest(n_candidates, 'combined_anomaly_score')
    
    # Create footprints
    footprints = []
    for idx, row in top_anomalies.iterrows():
        # Create a buffer around the point to form a bbox
        point_buffer = row.geometry.buffer(buffer_degrees)
        bbox = point_buffer.bounds  # (minx, miny, maxx, maxy)
        
        footprint = {
            'shot_id': row['shot_id'],
            'beam': row['beam'],
            'center_lat': row['latitude'],
            'center_lon': row['longitude'],
            'bbox_wkt': point_buffer.wkt,
            'height_anomaly': row['height_anomaly_score'],
            'pai_anomaly': row['pai_anomaly_score'],
            'combined_score': row['combined_anomaly_score']
        }
        footprints.append(footprint)
    
    return pd.DataFrame(footprints)

# Generate candidate footprints
print("Generating candidate footprints...")
candidate_footprints = generate_candidate_footprints(analysis_data)
print("\nTop 5 candidate anomaly footprints:")
print(candidate_footprints)

## Validate and Visualize Results

1. Convert candidate footprints to GeoDataFrame
2. Intersect with deforestation data
3. Calculate overlap statistics

In [ ]:
# Convert candidate footprints to GeoDataFrame
from shapely.wkt import loads

def validate_footprints(footprints, deforestation_data):
    # Convert WKT to geometry
    geometries = [loads(wkt) for wkt in footprints['bbox_wkt']]
    footprint_gdf = gpd.GeoDataFrame(
        footprints, 
        geometry=geometries,
        crs="EPSG:4326"
    )
    
    # Ensure same CRS
    if deforestation_data.crs != footprint_gdf.crs:
        deforestation_data = deforestation_data.to_crs(footprint_gdf.crs)
    
    # Check intersection with deforestation areas
    intersecting_areas = gpd.overlay(
        footprint_gdf, 
        deforestation_data, 
        how='intersection'
    )
    
    print(f"\nTotal candidate footprints: {len(footprint_gdf)}")
    print(f"Footprints intersecting with deforestation: {len(intersecting_areas)}")
    
    return footprint_gdf, intersecting_areas

# Validate results
footprint_gdf, intersecting_areas = validate_footprints(candidate_footprints, deforestation_data)

# Print detailed results
print("\nDetailed intersection results:")
for idx, row in footprint_gdf.iterrows():
    n_intersections = len(intersecting_areas[intersecting_areas.geometry.intersects(row.geometry)])
    print(f"\nFootprint {idx + 1}:")
    print(f"  Beam: {row['beam']}")
    print(f"  Combined Score: {row['combined_score']:.2f}")
    print(f"  Intersecting deforestation areas: {n_intersections}")

In [ ]:
# Save analysis metadata and results
analysis_results = {
    'timestamp': pd.Timestamp.now().isoformat(),
    'gedi_file': os.path.basename(gedi_path),
    'terrabrasilis_file': os.path.basename(terrabrasilis_path),
    'total_gedi_points': len(gedi_data),
    'filtered_points': len(processed_gedi),
    'anomaly_threshold': 0.95,
    'window_size': 5,
    'n_candidates': 5,
    'buffer_degrees': 0.01,
    'candidate_footprints': candidate_footprints.to_dict('records'),
    'intersecting_deforestation': len(intersecting_areas)
}

print("\nAnalysis Summary:")
print(f"Total GEDI points processed: {analysis_results['total_gedi_points']}")
print(f"Points after quality filtering: {analysis_results['filtered_points']}")
print(f"Candidate footprints generated: {len(analysis_results['candidate_footprints'])}")
print(f"Footprints intersecting deforestation: {analysis_results['intersecting_deforestation']}")


In [ ]:
# Save footprints to GeoJSON for reproducibility
import json

# Convert footprint_gdf to GeoJSON and save
footprint_gdf.to_file('anomaly_footprints.geojson', driver='GeoJSON')

# Save analysis parameters and results
with open('analysis_log.json', 'w') as f:
    json.dump(analysis_results, f, indent=4)

print("\nVerifying reproducibility:")
print("Loading saved footprints...")
loaded_footprints = gpd.read_file('anomaly_footprints.geojson')

# Compare original and loaded footprints
for i, (orig, loaded) in enumerate(zip(footprint_gdf.geometry, loaded_footprints.geometry)):
    distance = orig.centroid.distance(loaded.centroid) * 111000  # Convert to meters (approximate)
    print(f"Footprint {i+1} center distance: {distance:.2f} meters")
    if distance > 50:
        print(f"Warning: Distance exceeds 50m threshold!")


## Visualize Results

Create an interactive map showing:
1. Candidate anomaly footprints
2. Intersecting deforestation areas
3. GEDI measurement points colored by anomaly score

In [ ]:
# Install folium if not already installed
!pip install folium

In [ ]:


# import folium
# from folium import plugins

# # Create base map centered on the mean coordinates
# center_lat = processed_gedi['latitude'].mean()
# center_lon = processed_gedi['longitude'].mean()

# m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# # Sample points for visualization (10% of points)
# sample_size = len(analysis_data) // 10
# sampled_data = analysis_data.sample(n=sample_size, random_state=42)

# # Create a feature group for clustering
# marker_cluster = plugins.MarkerCluster().add_to(m)

# # Add sampled GEDI points with clustering
# for idx, row in sampled_data.iterrows():
#     color = f'rgb({int(255 * row.combined_anomaly_score)}, 0, 0)'
#     folium.CircleMarker(
#         location=[row.latitude, row.longitude],
#         radius=2,
#         color=color,
#         fill=True,
#         popup=f'Score: {row.combined_anomaly_score:.2f}'
#     ).add_to(marker_cluster)

# # Add candidate footprints
# for idx, row in footprint_gdf.iterrows():
#     folium.GeoJson(
#         row.geometry,
#         style_function=lambda x: {
#             'fillColor': 'yellow',
#             'color': 'red',
#             'weight': 2,
#             'fillOpacity': 0.3
#         },
#         popup=f'Footprint {idx+1}\nScore: {row.combined_score:.2f}'
#     ).add_to(m)

# # Add deforestation areas that intersect with footprints
# if len(intersecting_areas) > 0:
#     folium.GeoJson(
#         intersecting_areas,
#         style_function=lambda x: {
#             'fillColor': 'orange',
#             'color': 'black',
#             'weight': 1,
#             'fillOpacity': 0.5
#         },
#         popup=folium.GeoJsonPopup(fields=['year', 'class_name', 'area_km'])
#     ).add_to(m)

# # Add a legend
# legend_html = '''
# <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white; padding: 10px; border: 2px solid grey;">
#     <h4>Legend</h4>
#     <p><span style="color: red;">●</span> GEDI Points (color = anomaly score)</p>
#     <p><span style="color: yellow; background-color: rgba(255,255,0,0.3);">■</span> Candidate Footprints</p>
#     <p><span style="color: orange; background-color: rgba(255,165,0,0.5);">■</span> Deforestation Areas</p>
# </div>
# '''
# m.get_root().html.add_child(folium.Element(legend_html))

# # Display the map
# m

# Archaeological Analysis with GPT-4

Analyzing GEDI and TerraBrasilis data for potential archaeological significance:
1. Geometric patterns that might indicate human settlements
2. Relationships between canopy structure and potential archaeological features
3. Correlation with known archaeological sites
4. Historical context and Indigenous knowledge integration

In [ ]:
def analyze_archaeological_significance(footprints_df, deforestation_data):
    # Prepare the context for OpenAI
    site_descriptions = []
    for idx, row in footprints_df.iterrows():
        n_intersections = len(intersecting_areas[intersecting_areas.geometry.intersects(row.geometry)])
        desc = f"""Location {idx + 1}:
        Coordinates: Lat {row['center_lat']:.4f}, Lon {row['center_lon']:.4f}
        Canopy Height Anomaly: {row['height_anomaly']:.2f}
        Vegetation Density Variation: {row['pai_anomaly']:.2f}
        Observable Deforestation: {n_intersections} intersecting areas
        Geometric Pattern Score: {row['combined_score']:.2f}"""
        site_descriptions.append(desc)
    
    context = "\n".join(site_descriptions)
    
    prompt = f"""As an archaeological AI assistant analyzing potential ancient settlement sites in the Amazon rainforest, evaluate these locations identified through GEDI satellite LiDAR data and TerraBrasilis deforestation mapping:

{context}

Please provide a comprehensive analysis including:

1. Archaeological Significance:
   - Evaluate if any canopy patterns might indicate ancient human modifications
   - Analyze if height/density anomalies align with known patterns of archaeological sites
   - Identify potential geometric patterns that could suggest human settlement

2. Historical Context:
   - Compare these locations with known historical settlements in the region
   - Assess proximity to rivers or other features favored by ancient civilizations
   - Consider relationship to known trade routes or Indigenous pathways

3. Preservation Risk Assessment:
   - Evaluate immediate threats from deforestation
   - Suggest priority levels for archaeological investigation
   - Recommend conservation measures if needed

4. Investigation Recommendations:
   - Suggest specific types of follow-up remote sensing
   - Recommend ground verification methods
   - Propose additional data sources to cross-reference

5. Indigenous Knowledge Integration:
   - Suggest relevant Indigenous oral histories to consult
   - Identify potential connections to known Indigenous territories
   - Consider traditional land use patterns

Please focus on concrete, verifiable patterns and avoid speculation. Cite any relevant archaeological parallels or precedents."""

    client = OpenAI()
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )
        
        analysis = response.choices[0].message.content
        
        # Log the interaction
        log_openai_interaction(prompt, analysis)
        
        print("\nArchaeological Analysis:")
        print(analysis)
        
        return analysis
        
    except Exception as e:
        print(f"Error in OpenAI analysis: {str(e)}")
        return None

# Run archaeological analysis
print("\nPerforming archaeological analysis of anomalies...")
arch_analysis = analyze_archaeological_significance(footprint_gdf, deforestation_data)

# Update analysis results
analysis_results['archaeological_analysis'] = arch_analysis
analysis_results['analysis_timestamp'] = pd.Timestamp.now().isoformat()

# Save updated results
with open('archaeological_analysis_log.json', 'w') as f:
    json.dump(analysis_results, f, indent=4)

# Future Discovery and Research Directions

In [ ]:
def analyze_future_directions(footprints_df, deforestation_data, analysis_results):
    # Prepare comprehensive context
    site_context = {
        'anomaly_sites': footprints_df[['center_lat', 'center_lon', 'height_anomaly', 'pai_anomaly', 'combined_score']].to_dict('records'),
        'deforestation_patterns': len(intersecting_areas),
        'total_analyzed_points': analysis_results['total_gedi_points'],
        'analysis_timestamp': analysis_results['timestamp']
    }
    
    # Create a detailed prompt for future research directions
    prompt = f"""As an AI research assistant specializing in Amazonian archaeology and remote sensing, analyze our findings and suggest future research directions. Here's our current data:

Summary of Findings:
- Analyzed {site_context['total_analyzed_points']} GEDI LiDAR points
- Identified {len(site_context['anomaly_sites'])} high-priority sites
- Found {site_context['deforestation_patterns']} intersecting deforestation areas
- Timestamp: {site_context['analysis_timestamp']}

Anomaly Site Details:
{json.dumps(site_context['anomaly_sites'], indent=2)}

Please provide a comprehensive research strategy addressing:

1. Cross-Referencing Opportunities:
   - Identify complementary satellite/LiDAR datasets
   - Suggest historical maps and documents to consult
   - List relevant archaeological databases

2. Advanced Analysis Methods:
   - Propose machine learning approaches for pattern detection
   - Suggest temporal analysis techniques
   - Recommend geometric analysis methods

3. New Research Hypotheses:
   - Generate testable hypotheses about site functions
   - Propose settlement pattern theories
   - Suggest connections to known archaeological features

4. Collaboration Opportunities:
   - Identify relevant research institutions
   - Suggest Indigenous communities to consult
   - List potential interdisciplinary connections

5. Technology Integration:
   - Recommend additional remote sensing technologies
   - Suggest data fusion approaches
   - Propose visualization techniques

Focus on concrete, actionable research steps that could lead to significant archaeological discoveries."""

    client = OpenAI()
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )
        
        future_directions = response.choices[0].message.content
        
        # Log the interaction
        log_openai_interaction(prompt, future_directions)
        
        # Save to a dedicated research directions file
        research_plan = {
            'timestamp': datetime.now().isoformat(),
            'input_context': site_context,
            'ai_analysis': future_directions
        }
        
        with open('research_directions.json', 'w') as f:
            json.dump(research_plan, f, indent=4)
        
        print("\nFuture Research Directions Analysis:")
        print(future_directions)
        
        return future_directions
        
    except Exception as e:
        print(f"Error in future directions analysis: {str(e)}")
        return None

# Run future directions analysis
print("\nAnalyzing future research directions...")
future_analysis = analyze_future_directions(footprint_gdf, deforestation_data, analysis_results)